In [22]:
import numpy as np
from RK_bin import *
import matplotlib.pyplot as plt
import matplotlib as mpl

from IPython.display import clear_output
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
#mpl.use("pgf")
preamble = r"""
\usepackage{bm}
\renewcommand{\vec}{\bm}
"""

mpl.use("pgf")


mpl.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'font.size' : 8,
    'text.usetex': True,
    'pgf.rcfonts': False,
    'pgf.preamble': preamble
})

In [24]:
d = 100
n = 1000000

np.random.seed(0)

U,_ = np.linalg.qr(np.random.randn(n,d))
V,_ = np.linalg.qr(np.random.randn(d,d))

In [25]:
np.random.seed(0)

κ = 50

Λ = 1+np.linspace(0,1,d)*(κ-1)*.5**(d-np.arange(d)-1)

Σ = np.diag(np.sqrt(Λ))    
A = U@Σ@V
x_opt = np.random.randn(d)
r = np.random.randn(n)
b = A@x_opt + 1e-5*r/np.linalg.norm(r)
x_opt = np.linalg.lstsq(A,b,rcond=None)[0]

λ,ℓ,L,κ,κ_,α,β,η,κC = get_params(A,c=1e3)

In [26]:
k_max = 100

B = getB(None,α,β,λ,κC,mode='approx')

n_trials = 100

B_scales = [1e1,1e0,1e-1]


In [27]:
err_HBM = []
res_HBM = []

x_HBM = HBM(A,b,k_max,α,β)
err_HBM.append(np.linalg.norm(x_HBM - x_opt[None,:],axis=1))
res_HBM.append(np.linalg.norm(b-(A@x_HBM.T).T,axis=1))

In [6]:
err_RK = np.zeros((n_trials,k_max))
res_RK = np.zeros((n_trials,k_max))

output_skip = 50
np.random.seed(0)
for j in range(n_trials):
    print(f'trial {j}/{n_trials}')
    clear_output(wait=True)
    
    x_RK = RK(A,b,k_max,output_skip=output_skip)
    err_RK[j] = np.linalg.norm(x_RK - x_opt[None,:],axis=1)
    res_RK[j] = np.linalg.norm(b-(A@x_RK.T).T,axis=1)

trial 99/100


In [9]:
err_HBM_mb = []
res_HBM_mb = []
for scale in B_scales:
        
    err_HBM_mb_expr = np.zeros((n_trials,k_max))
    res_HBM_mb_expr = np.zeros((n_trials,k_max))
    np.random.seed(0)
    for j in range(n_trials):
        print(f'{scale}, {int(B*scale)}, trial {j}/{n_trials}')
        clear_output(wait=True)

        x_HBM_mb = minibatch_HBM(A,b,k_max,α,β,int(B*scale),sampling='row_norm')
        err_HBM_mb_expr[j] = np.linalg.norm(x_HBM_mb - x_opt[None,:],axis=1)
        res_HBM_mb_expr[j] = np.linalg.norm(b-(A@x_HBM_mb.T).T,axis=1)
        
    err_HBM_mb.append(err_HBM_mb_expr)
    res_HBM_mb.append(res_HBM_mb_expr)

0.1, 2657, trial 99/100


In [28]:
#np.save('data/inconsistent.npy',[err_RK,res_RK,err_HBM_mb,res_HBM_mb],allow_pickle=True)
[err_RK,res_RK,err_HBM_mb,res_HBM_mb] = np.load('data/inconsistent.npy',allow_pickle=True)

In [37]:
colors = ['#1f4c84','#800000','#979700','#007f68']
line_styles = ['--','-.',(0, (4, 2, 1, 1, 1, 2)),':']

colors = colors[-1:]+colors[:-1]
line_styles = line_styles[-1:]+line_styles[:-1]

In [38]:
fig,axs = plt.subplots(1,2,figsize=(8,3.5),sharex='col',sharey=True)

plt.subplots_adjust(wspace=.05)
plt.subplots_adjust(left=0.05, right=0.95, top=0.9, bottom=.2)

rate = np.sqrt(2)*κC*np.sqrt(β)**np.arange(k_max)
#axs[0].plot(np.arange(k_max),rate,\
#           color='k',ls=':',lw=1)
axs[1].axhline(np.linalg.norm(b-A@x_opt)/np.linalg.norm(b),\
            color='k',ls=':',lw=1,label=r'$\|\vec{b}-\vec{A}\vec{x}^*\|$')

axs[0].plot(np.arange(k_max),err_HBM[0]/err_HBM[0][0],\
            color='k',ls='-',lw=1,label='HBM')
axs[1].plot(np.arange(k_max),res_HBM[0]/res_HBM[0][0],\
            color='k',ls='-',lw=1,label='HBM')

σ = .05
        
median = np.quantile(err_RK/err_RK[0,0],.5,axis=0)
upper = np.quantile(err_RK/err_RK[0,0],1.-σ,axis=0)
lower = np.quantile(err_RK/err_RK[0,0],σ,axis=0)

axs[0].plot(np.arange(k_max),median,\
           color=colors[-1],ls=line_styles[-1],label='RK')
axs[0].fill_between(np.arange(k_max),lower,upper,alpha=.15,\
            color=colors[-1],ls=line_styles[-1])

axs[1].plot(np.arange(k_max),np.median(res_RK,axis=0)/res_RK[0][0],\
           color=colors[-1],ls=line_styles[-1],label='RK')

for j,scale in enumerate(B_scales):

    median = np.quantile(err_HBM_mb[j]/err_HBM_mb[j][0,0],.5,axis=0)
    upper = np.quantile(err_HBM_mb[j]/err_HBM_mb[j][0,0],1.-σ,axis=0)
    lower = np.quantile(err_HBM_mb[j]/err_HBM_mb[j][0,0],σ,axis=0)
        
    axs[0].plot(np.arange(k_max),median,\
            color=colors[j],ls=line_styles[j])
    axs[0].fill_between(np.arange(k_max),lower,upper,alpha=.15,\
            color=colors[j],ls=line_styles[j])
    
    median = np.quantile(res_HBM_mb[j]/res_HBM_mb[j][0,0],.5,axis=0)
    upper = np.quantile(res_HBM_mb[j]/res_HBM_mb[j][0,0],1.-σ,axis=0)
    lower = np.quantile(res_HBM_mb[j]/res_HBM_mb[j][0,0],σ,axis=0)
    
    axs[1].plot(np.arange(k_max),median,\
            color=colors[j],ls=line_styles[j],label=f'$B={scale}B^*$')
    axs[1].fill_between(np.arange(k_max),lower,upper,alpha=.15,\
            color=colors[j],ls=line_styles[j])
    
axs[0].set_title(r'error: $\|\vec{x}_k-\vec{x}^*\|$')
axs[1].set_title(r'residual: $\|\vec{b}-\vec{A}\vec{x}_k\|$')

axs[0].set_yscale('log')
axs[0].set_ylim(1e-10,1e1)

axs[0].set_xlabel(f'iteration: $k$ ($k/{output_skip}$ for RK)')
axs[1].set_xlabel(f'iteration: $k$ ($k/{output_skip}$ for RK)')

axs[1].legend(loc='upper center', bbox_to_anchor=(0,-.17), ncol=7)


plt.savefig('imgs/inconsistent.pdf')
plt.close()